# Load Models

In [39]:
import torch

# Define the paths to the models
bloombert_path = 'model/bloombert_model.pt'
sembert_path = 'model/obert_distilbert.pt'

# Load the models
try:
    bloombert_model_weights = torch.load(bloombert_path)
    obert_distilbert_weights = torch.load(sembert_path)
    print("Models loaded successfully!")
except FileNotFoundError:
    print("Error: Make sure the model files exist at the specified paths.")
except Exception as e:
    print(f"An error occurred while loading the models: {e}")

Models loaded successfully!


# Load Data

In [40]:
import pandas as pd

df = pd.read_csv('dataset/test/test.csv')
display(df.head())

,Learning Outcomes,Deliverables,Assessments
0,Discuss and demonstrate different ways how Jav...,Learning Log\nLinkedIn Learning Certificate/ C...,Group Exercise Case Study/Analysis
1,Fundamentals of pricing\nTotal Cost of Ownersh...,Case Study Presentation,Knowledge Check\nCase Study (Support Plan)
2,Presentation the output of the Project,Final Project,Updated project in GitHub Updated Project Shar...
3,Discuss different cases and implementations of...,Security Articles\nLinkedIn Learning Certifica...,Group Exercise Case Study/Analysis
4,Explain the overview of computers and programm...,Completed lab/written exercises. \nCompleted quiz,Recitation \nQuiz \nChapter exercises


# Data Prep

In [41]:
columns_to_extract = ['Learning Outcomes', 'Deliverables', 'Assessments']
data_list = []
for col in columns_to_extract:
    data_list.extend(df[col].astype(str).tolist())

# Semantic Alignment

# Calculate for Semantic Similarity

In [42]:
from transformers import DistilBertModel, DistilBertTokenizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import torch

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')

# Load the pre-trained weights
# Adjust keys to match the model's state_dict
adjusted_weights = {k.replace('0.auto_model.', ''): v for k, v in obert_distilbert_weights.items() if '0.auto_model.' in k}
model.load_state_dict(adjusted_weights)


# Tokenize the data
encoded_input = tokenizer(data_list, padding=True, truncation=True, return_tensors='pt', max_length=512)

# Get embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

# Using the last hidden state as embeddings
embeddings = model_output.last_hidden_state[:, 0, :]

# Calculate cosine similarity
similarity_matrix = cosine_similarity(embeddings)

display(similarity_matrix)

array([[1.0000001 , 0.3058113 , 0.6070606 , ..., 0.3464562 , 0.3887711 ,
        0.46139008],
       [0.3058113 , 1.        , 0.4023333 , ..., 0.43953884, 0.26708978,
        0.44953975],
       [0.6070606 , 0.4023333 , 1.        , ..., 0.3173246 , 0.50836176,
        0.6778221 ],
       ...,
       [0.3464562 , 0.43953884, 0.3173246 , ..., 0.99999976, 0.1712772 ,
        0.4079083 ],
       [0.3887711 , 0.26708978, 0.50836176, ..., 0.1712772 , 1.        ,
        0.5275333 ],
       [0.46139008, 0.44953975, 0.6778221 , ..., 0.4079083 , 0.5275333 ,
        0.9999999 ]], shape=(36, 36), dtype=float32)

In [43]:
import numpy as np
import pandas as pd

num_rows = len(df)
similarity_scores = []

for i in range(num_rows):
    lo_deliverables_sim = similarity_matrix[i, i + num_rows]
    lo_assessments_sim = similarity_matrix[i, i + 2 * num_rows]
    deliverables_assessments_sim = similarity_matrix[i + num_rows, i + 2 * num_rows]
    # Calculate the average of the three similarities for each row
    overall_alignment_score = (lo_deliverables_sim + lo_assessments_sim + deliverables_assessments_sim) / 3
    similarity_scores.append([lo_deliverables_sim, lo_assessments_sim, deliverables_assessments_sim, overall_alignment_score])

# DataFrame to display the pairwise and overall similarity scores for each row
similarity_df = pd.DataFrame(similarity_scores, columns=['LO-Deliverables Similarity', 'LO-Assessments Similarity', 'Deliverables-Assessments Similarity', 'Overall Alignment Score'])

display(similarity_df)

,LO-Deliverables Similarity,LO-Assessments Similarity,Deliverables-Assessments Similarity,Overall Alignment Score
0,0.588335,0.461390,0.554043,0.534589
1,0.419397,0.504238,0.874965,0.599533
2,0.884119,0.719980,0.702764,0.768954
3,0.508374,0.703519,0.571918,0.594603
4,0.452947,0.428460,0.841808,0.574405
5,0.191211,0.290230,0.509248,0.330230
6,0.438671,0.620152,0.744928,0.601250
7,0.356866,0.652357,0.702788,0.570671
8,0.177504,0.405143,0.582871,0.388506
9,0.762243,0.762268,0.889574,0.804695


In [58]:
# Add an 'Alignment' column
similarity_df['Semantic Alignment'] = np.where(similarity_df['Overall Alignment Score'] >= 0.6, 'Aligned', 'Misaligned')

display(similarity_df)

,LO-Deliverables Similarity,LO-Assessments Similarity,Deliverables-Assessments Similarity,Overall Alignment Score,Alignment,Semantic Alignment
0,0.588335,0.461390,0.554043,0.534589,Misaligned,Misaligned
1,0.419397,0.504238,0.874965,0.599533,Misaligned,Misaligned
2,0.884119,0.719980,0.702764,0.768954,Aligned,Aligned
3,0.508374,0.703519,0.571918,0.594603,Misaligned,Misaligned
4,0.452947,0.428460,0.841808,0.574405,Misaligned,Misaligned
5,0.191211,0.290230,0.509248,0.330230,Misaligned,Misaligned
6,0.438671,0.620152,0.744928,0.601250,Aligned,Aligned
7,0.356866,0.652357,0.702788,0.570671,Misaligned,Misaligned
8,0.177504,0.405143,0.582871,0.388506,Misaligned,Misaligned
9,0.762243,0.762268,0.889574,0.804695,Aligned,Aligned


# Bloom's Taxonomy Alignment

# Load BloomBERT

In [45]:
from transformers import DistilBertForSequenceClassification

bloombert_model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=6)

adjusted_bloombert_weights = {k.replace('bert.', 'distilbert.'): v for k, v in bloombert_model_weights.items() if 'bert.' in k}

bloombert_model.load_state_dict(adjusted_bloombert_weights, strict=False)

display(bloombert_model)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


# Category Map

In [46]:
category_map = {0: 'Remembering', 1: 'Understanding', 2: 'Applying', 3: 'Analyzing', 4: 'Evaluating', 5: 'Creating'}

# Tokenize Data for BloomBERT to Classify

In [47]:
bloombert_input_data = []
for index, row in df[columns_to_extract].iterrows():
    bloombert_input_data.extend(row.astype(str).tolist())

display(bloombert_input_data[:10])

['Discuss and demonstrate different ways how JavaScript can be used for Penetration Testing. Payload Creation\nGithub repositories\nTools based on JavaScript\nTool customization',
 'Learning Log\nLinkedIn Learning Certificate/ Cybrary Certificate',
 'Group Exercise Case Study/Analysis',
 'Fundamentals of pricing\nTotal Cost of Ownership\nAWS Organizations\nAWS Billing and Cost Management\nTechnical Support',
 'Case Study Presentation',
 'Knowledge Check\nCase Study (Support Plan)',
 'Presentation the output of the Project',
 'Final Project',
 'Updated project in GitHub Updated Project SharePoint Site\nFinal Project Presntation (Face to Face)',
 'Discuss different cases and implementations of successful and failed BCPs.']

In [48]:
encoded_bloombert_input = tokenizer(bloombert_input_data, padding=True, truncation=True, return_tensors='pt', max_length=512)

with torch.no_grad():
    bloombert_output = bloombert_model(**encoded_bloombert_input)

logits = bloombert_output.logits

threshold = 0.0
predicted_classes_multi = (logits > threshold).nonzero(as_tuple=True)[1]

predicted_levels_indices = []
start_index = 0
for i in range(len(bloombert_input_data)):
    text_logits = logits[i]
    predicted_indices_for_text = (text_logits > threshold).nonzero(as_tuple=True)[0].tolist()
    predicted_levels_indices.append(predicted_indices_for_text)


display(predicted_levels_indices)

[[4],
 [0, 2, 3],
 [3, 4],
 [0, 2, 3, 4],
 [3],
 [3],
 [0, 2, 3, 4],
 [0],
 [2, 3],
 [2],
 [2, 3],
 [3, 4],
 [2],
 [0, 3],
 [0, 2, 3],
 [2, 3],
 [0, 3],
 [0, 2, 3],
 [0, 2, 3],
 [3],
 [2, 3],
 [0, 2, 3],
 [3],
 [0, 3],
 [0, 2, 3],
 [0, 3],
 [0, 2, 3],
 [3, 4],
 [3],
 [],
 [],
 [2, 4],
 [2, 3, 4],
 [0],
 [0, 2, 3],
 [3, 4]]

# Predict the Blooms Levels Used

In [27]:
predicted_bloom_levels_multi = []
for text_indices in predicted_levels_indices:
    mapped_levels = [category_map[idx] for idx in text_indices]
    predicted_bloom_levels_multi.append(mapped_levels)

display(predicted_bloom_levels_multi)

[['Understanding', 'Analyzing', 'Evaluating', 'Creating'],
 ['Analyzing', 'Evaluating', 'Creating'],
 ['Remembering',
  'Understanding',
  'Applying',
  'Analyzing',
  'Evaluating',
  'Creating'],
 ['Remembering', 'Understanding', 'Analyzing', 'Evaluating', 'Creating'],
 ['Understanding', 'Analyzing', 'Evaluating', 'Creating'],
 ['Remembering', 'Understanding', 'Analyzing', 'Evaluating', 'Creating'],
 ['Remembering', 'Understanding', 'Analyzing', 'Evaluating', 'Creating'],
 ['Understanding', 'Evaluating', 'Creating'],
 ['Understanding', 'Analyzing', 'Evaluating', 'Creating'],
 ['Understanding', 'Analyzing', 'Evaluating', 'Creating'],
 ['Understanding', 'Analyzing', 'Evaluating', 'Creating'],
 ['Remembering',
  'Understanding',
  'Applying',
  'Analyzing',
  'Evaluating',
  'Creating'],
 ['Understanding', 'Analyzing', 'Evaluating', 'Creating'],
 ['Remembering', 'Understanding', 'Evaluating', 'Creating'],
 ['Understanding', 'Analyzing', 'Evaluating', 'Creating'],
 ['Applying', 'Analyzing

# Display the Predicted Bloom's Level

In [49]:
df['LO Bloom Levels'] = predicted_bloom_levels_multi[0:num_rows]
df['Deliverable Bloom Levels'] = predicted_bloom_levels_multi[num_rows:2*num_rows]
df['Assessment Bloom Levels'] = predicted_bloom_levels_multi[2*num_rows:3*num_rows]

display(df)

,Learning Outcomes,Deliverables,Assessments,LO Bloom Levels,Deliverable Bloom Levels,Assessment Bloom Levels
0,Discuss and demonstrate different ways how Jav...,Learning Log\nLinkedIn Learning Certificate/ C...,Group Exercise Case Study/Analysis,"[Understanding, Analyzing, Evaluating, Creating]","[Understanding, Analyzing, Evaluating, Creating]","[Understanding, Applying, Analyzing, Evaluatin..."
1,Fundamentals of pricing\nTotal Cost of Ownersh...,Case Study Presentation,Knowledge Check\nCase Study (Support Plan),"[Analyzing, Evaluating, Creating]","[Remembering, Understanding, Evaluating, Creat...","[Understanding, Analyzing, Evaluating, Creating]"
2,Presentation the output of the Project,Final Project,Updated project in GitHub Updated Project Shar...,"[Remembering, Understanding, Applying, Analyzi...","[Understanding, Analyzing, Evaluating, Creating]","[Understanding, Analyzing, Evaluating, Creating]"
3,Discuss different cases and implementations of...,Security Articles\nLinkedIn Learning Certifica...,Group Exercise Case Study/Analysis,"[Remembering, Understanding, Analyzing, Evalua...","[Applying, Analyzing, Evaluating, Creating]","[Understanding, Applying, Analyzing, Evaluatin..."
4,Explain the overview of computers and programm...,Completed lab/written exercises. \nCompleted quiz,Recitation \nQuiz \nChapter exercises,"[Understanding, Analyzing, Evaluating, Creating]","[Remembering, Understanding, Evaluating, Creat...","[Remembering, Understanding, Analyzing, Evalua..."
5,Define and use arrays and describe how they oc...,Code Combat CS4 Completion Certificate\nComple...,Recitation \nLab/Written Exercises \nChapter e...,"[Remembering, Understanding, Analyzing, Evalua...","[Remembering, Understanding, Analyzing, Evalua...","[Understanding, Applying, Analyzing, Evaluatin..."
6,Amazon Elastic Block Store (Amazon EBS)\nAmazo...,Case Study Presentation,Knowledge Check\nLab 6\nCase Study (Storage So...,"[Remembering, Understanding, Analyzing, Evalua...","[Remembering, Understanding, Analyzing, Evalua...","[Understanding, Analyzing, Evaluating, Creating]"
7,Amazon Relational Database Service (Amazon RDS...,Case Study Presentation,Knowledge Check Lab 7\nCase Study (Database So...,"[Understanding, Evaluating, Creating]","[Understanding, Analyzing, Evaluating, Creating]","[Understanding, Analyzing, Evaluating, Creating]"
8,Explore input data for analysis\nApply basic f...,LinkedIn Learning Certificate,In-video quizzes\nProblem solving exercise,"[Understanding, Analyzing, Evaluating, Creating]","[Remembering, Understanding, Analyzing, Evalua...","[Understanding, Analyzing, Evaluating, Creating]"
9,Determine ways in which the three key elements...,Certificates:\nModule 1: Data and Privacy\nMod...,Assessment: Show me what you know:\nModule 1: ...,"[Understanding, Analyzing, Evaluating, Creating]","[Understanding, Evaluating, Creating]","[Understanding, Analyzing, Evaluating, Creating]"


# Check for Bloom's Verbs in Learning Outcomes

# Load Learning Outcomes to Begin Checking for Bloom's Verbs

In [29]:
learning_outcomes_text = df['Learning Outcomes'].astype(str).tolist()

display(learning_outcomes_text[:10])

['Discuss and demonstrate different ways how JavaScript can be used for Penetration Testing. Payload Creation\nGithub repositories\nTools based on JavaScript\nTool customization',
 'Fundamentals of pricing\nTotal Cost of Ownership\nAWS Organizations\nAWS Billing and Cost Management\nTechnical Support',
 'Presentation the output of the Project',
 'Discuss different cases and implementations of successful and failed BCPs.',
 'Explain the overview of computers and programming.',
 'Define and use arrays and describe how they occupy computer memory.\xa0\nManipulate arrays to replace nested decisions.\xa0\nIdentify the size of an array and recognize the notation used to identify an element of an array.\xa0\nDefine parallel arrays.\xa0\nManipulate arrays to replace nested decisions.\xa0\nCreate a flowchart that uses a - for loop to process arrays.',
 'Amazon Elastic Block Store (Amazon EBS)\nAmazon Simple Storage Service (Amazon S3)\nAmazon Elastic File System (Amazon EFS)\nAmazon Simple Stor

# Load CSV file that contains the Verbs

In [50]:
import pandas as pd

# Load the CSV file
blooms_verbs_df = pd.read_csv('dataset/blooms_verbs/blooms_verbs.csv')

# Assuming the CSV has one column with verb names, extract them into a set for efficient lookup
if not blooms_verbs_df.empty:
    blooms_verbs_set = set(blooms_verbs_df.iloc[:, 0].astype(str).str.lower().tolist())
else:
    blooms_verbs_set = set()

display(blooms_verbs_set)

{'cite',
 'define',
 'describe',
 'draw',
 'enumerate',
 'identify',
 'index',
 'indicate',
 'label',
 'list',
 'match',
 'meet',
 'name',
 'nan',
 'outline',
 'point',
 'quote',
 'read',
 'recall',
 'recite',
 'recognize',
 'record',
 'repeat',
 'reproduce',
 'review',
 'select',
 'state',
 'study',
 'tabulate',
 'trace',
 'write'}

# Extract and Preprocess Learning Outcomes

In [51]:
learning_outcomes_text_processed = df['Learning Outcomes'].astype(str).str.lower().str.findall(r'\b\w+\b').tolist()

display(learning_outcomes_text_processed[:10])

[['discuss',
  'and',
  'demonstrate',
  'different',
  'ways',
  'how',
  'javascript',
  'can',
  'be',
  'used',
  'for',
  'penetration',
  'testing',
  'payload',
  'creation',
  'github',
  'repositories',
  'tools',
  'based',
  'on',
  'javascript',
  'tool',
  'customization'],
 ['fundamentals',
  'of',
  'pricing',
  'total',
  'cost',
  'of',
  'ownership',
  'aws',
  'organizations',
  'aws',
  'billing',
  'and',
  'cost',
  'management',
  'technical',
  'support'],
 ['presentation', 'the', 'output', 'of', 'the', 'project'],
 ['discuss',
  'different',
  'cases',
  'and',
  'implementations',
  'of',
  'successful',
  'and',
  'failed',
  'bcps'],
 ['explain', 'the', 'overview', 'of', 'computers', 'and', 'programming'],
 ['define',
  'and',
  'use',
  'arrays',
  'and',
  'describe',
  'how',
  'they',
  'occupy',
  'computer',
  'memory',
  'manipulate',
  'arrays',
  'to',
  'replace',
  'nested',
  'decisions',
  'identify',
  'the',
  'size',
  'of',
  'an',
  'array'

# Check for Bloom's Verbs

In [52]:
bloom_verb_detection_status_csv = []
for word_list in learning_outcomes_text_processed:
    detected = any(word in blooms_verbs_set for word in word_list)
    bloom_verb_detection_status_csv.append('Detected' if detected else 'Undetected')

display(bloom_verb_detection_status_csv[:10])

['Undetected',
 'Undetected',
 'Undetected',
 'Undetected',
 'Undetected',
 'Detected',
 'Undetected',
 'Undetected',
 'Undetected',
 'Undetected']

In [53]:
df['Blooms Verb'] = bloom_verb_detection_status_csv

display(df.head())

,Learning Outcomes,Deliverables,Assessments,LO Bloom Levels,Deliverable Bloom Levels,Assessment Bloom Levels,Blooms Verb
0,Discuss and demonstrate different ways how Jav...,Learning Log\nLinkedIn Learning Certificate/ C...,Group Exercise Case Study/Analysis,"[Understanding, Analyzing, Evaluating, Creating]","[Understanding, Analyzing, Evaluating, Creating]","[Understanding, Applying, Analyzing, Evaluatin...",Undetected
1,Fundamentals of pricing\nTotal Cost of Ownersh...,Case Study Presentation,Knowledge Check\nCase Study (Support Plan),"[Analyzing, Evaluating, Creating]","[Remembering, Understanding, Evaluating, Creat...","[Understanding, Analyzing, Evaluating, Creating]",Undetected
2,Presentation the output of the Project,Final Project,Updated project in GitHub Updated Project Shar...,"[Remembering, Understanding, Applying, Analyzi...","[Understanding, Analyzing, Evaluating, Creating]","[Understanding, Analyzing, Evaluating, Creating]",Undetected
3,Discuss different cases and implementations of...,Security Articles\nLinkedIn Learning Certifica...,Group Exercise Case Study/Analysis,"[Remembering, Understanding, Analyzing, Evalua...","[Applying, Analyzing, Evaluating, Creating]","[Understanding, Applying, Analyzing, Evaluatin...",Undetected
4,Explain the overview of computers and programm...,Completed lab/written exercises. \nCompleted quiz,Recitation \nQuiz \nChapter exercises,"[Understanding, Analyzing, Evaluating, Creating]","[Remembering, Understanding, Evaluating, Creat...","[Remembering, Understanding, Analyzing, Evalua...",Undetected


# Check for Bloom's Alignment

# Dictate Bloom's Hierarchy

In [54]:
bloom_hierarchy = {
    'Remembering': 1,
    'Understanding': 2,
    'Applying': 3,
    'Analyzing': 4,
    'Evaluating': 5,
    'Creating': 6
}

display(bloom_hierarchy)

{'Remembering': 1,
 'Understanding': 2,
 'Applying': 3,
 'Analyzing': 4,
 'Evaluating': 5,
 'Creating': 6}

# Check for Bloom's Alignment

In [55]:
bloom_alignment = []

for index, row in df.iterrows():
    blooms_verb = row['Blooms Verb']
    lo_bloom_levels = row['LO Bloom Levels']
    del_bloom_levels = row['Deliverable Bloom Levels']
    ass_bloom_levels = row['Assessment Bloom Levels']

    if blooms_verb == 'Detected':
        if len(lo_bloom_levels) == 1:
            # If there is only 1 Bloom's Level in 'LO' then 'DEL' and 'ASS' must be greater than or equal to the Bloom's Level of 'LO'
            lo_level = bloom_hierarchy.get(lo_bloom_levels[0], 0)
            del_aligned = all(bloom_hierarchy.get(level, 0) >= lo_level for level in del_bloom_levels) if del_bloom_levels else False
            ass_aligned = all(bloom_hierarchy.get(level, 0) >= lo_level for level in ass_bloom_levels) if ass_bloom_levels else False

            if del_aligned and ass_aligned:
                bloom_alignment.append('Aligned')
            else:
                bloom_alignment.append('Misaligned')
        elif len(lo_bloom_levels) >= 2:
            # If there are 2 or more Bloom's Levels in 'LO'
            # 'DEL' must be greater than or equal to the lowest Bloom's Level of 'LO'
            # 'ASS' must be greater than or equal to the highest Bloom's Level of 'LO'
            lowest_lo_level = min(bloom_hierarchy.get(level, 0) for level in lo_bloom_levels)
            highest_lo_level = max(bloom_hierarchy.get(level, 0) for level in lo_bloom_levels)

            del_aligned = any(bloom_hierarchy.get(level, 0) >= lowest_lo_level for level in del_bloom_levels) if del_bloom_levels else False
            ass_aligned = any(bloom_hierarchy.get(level, 0) >= highest_lo_level for level in ass_bloom_levels) if ass_bloom_levels else False

            if del_aligned and ass_aligned:
                bloom_alignment.append('Aligned')
            else:
                bloom_alignment.append('Misaligned')
        else:
            # No Bloom levels detected in LO, misaligned
            bloom_alignment.append('Misaligned')

    else:
        # 'Blooms Verb' must be 'Detected'
        bloom_alignment.append('Misaligned')

# Add the 'Bloom Alignment' column to the DataFrame
df['Bloom Alignment'] = bloom_alignment

# Display the final DataFrame
display(df)

,Learning Outcomes,Deliverables,Assessments,LO Bloom Levels,Deliverable Bloom Levels,Assessment Bloom Levels,Blooms Verb,Bloom Alignment
0,Discuss and demonstrate different ways how Jav...,Learning Log\nLinkedIn Learning Certificate/ C...,Group Exercise Case Study/Analysis,"[Understanding, Analyzing, Evaluating, Creating]","[Understanding, Analyzing, Evaluating, Creating]","[Understanding, Applying, Analyzing, Evaluatin...",Undetected,Misaligned
1,Fundamentals of pricing\nTotal Cost of Ownersh...,Case Study Presentation,Knowledge Check\nCase Study (Support Plan),"[Analyzing, Evaluating, Creating]","[Remembering, Understanding, Evaluating, Creat...","[Understanding, Analyzing, Evaluating, Creating]",Undetected,Misaligned
2,Presentation the output of the Project,Final Project,Updated project in GitHub Updated Project Shar...,"[Remembering, Understanding, Applying, Analyzi...","[Understanding, Analyzing, Evaluating, Creating]","[Understanding, Analyzing, Evaluating, Creating]",Undetected,Misaligned
3,Discuss different cases and implementations of...,Security Articles\nLinkedIn Learning Certifica...,Group Exercise Case Study/Analysis,"[Remembering, Understanding, Analyzing, Evalua...","[Applying, Analyzing, Evaluating, Creating]","[Understanding, Applying, Analyzing, Evaluatin...",Undetected,Misaligned
4,Explain the overview of computers and programm...,Completed lab/written exercises. \nCompleted quiz,Recitation \nQuiz \nChapter exercises,"[Understanding, Analyzing, Evaluating, Creating]","[Remembering, Understanding, Evaluating, Creat...","[Remembering, Understanding, Analyzing, Evalua...",Undetected,Misaligned
5,Define and use arrays and describe how they oc...,Code Combat CS4 Completion Certificate\nComple...,Recitation \nLab/Written Exercises \nChapter e...,"[Remembering, Understanding, Analyzing, Evalua...","[Remembering, Understanding, Analyzing, Evalua...","[Understanding, Applying, Analyzing, Evaluatin...",Detected,Aligned
6,Amazon Elastic Block Store (Amazon EBS)\nAmazo...,Case Study Presentation,Knowledge Check\nLab 6\nCase Study (Storage So...,"[Remembering, Understanding, Analyzing, Evalua...","[Remembering, Understanding, Analyzing, Evalua...","[Understanding, Analyzing, Evaluating, Creating]",Undetected,Misaligned
7,Amazon Relational Database Service (Amazon RDS...,Case Study Presentation,Knowledge Check Lab 7\nCase Study (Database So...,"[Understanding, Evaluating, Creating]","[Understanding, Analyzing, Evaluating, Creating]","[Understanding, Analyzing, Evaluating, Creating]",Undetected,Misaligned
8,Explore input data for analysis\nApply basic f...,LinkedIn Learning Certificate,In-video quizzes\nProblem solving exercise,"[Understanding, Analyzing, Evaluating, Creating]","[Remembering, Understanding, Analyzing, Evalua...","[Understanding, Analyzing, Evaluating, Creating]",Undetected,Misaligned
9,Determine ways in which the three key elements...,Certificates:\nModule 1: Data and Privacy\nMod...,Assessment: Show me what you know:\nModule 1: ...,"[Understanding, Analyzing, Evaluating, Creating]","[Understanding, Evaluating, Creating]","[Understanding, Analyzing, Evaluating, Creating]",Undetected,Misaligned


# Constructive Alignment Checker

In [60]:
df['Semantic Alignment'] = similarity_df['Semantic Alignment']

df['Overall Final Alignment'] = np.where(
    (df['Semantic Alignment'] == 'Aligned') & (df['Bloom Alignment'] == 'Aligned'),
    'Aligned',
    'Misaligned'
)

display(df[['Learning Outcomes', 'Deliverables', 'Assessments', 'Semantic Alignment', 'Bloom Alignment', 'Overall Final Alignment']])

,Learning Outcomes,Deliverables,Assessments,Semantic Alignment,Bloom Alignment,Overall Final Alignment
0,Discuss and demonstrate different ways how Jav...,Learning Log\nLinkedIn Learning Certificate/ C...,Group Exercise Case Study/Analysis,Misaligned,Misaligned,Misaligned
1,Fundamentals of pricing\nTotal Cost of Ownersh...,Case Study Presentation,Knowledge Check\nCase Study (Support Plan),Misaligned,Misaligned,Misaligned
2,Presentation the output of the Project,Final Project,Updated project in GitHub Updated Project Shar...,Aligned,Misaligned,Misaligned
3,Discuss different cases and implementations of...,Security Articles\nLinkedIn Learning Certifica...,Group Exercise Case Study/Analysis,Misaligned,Misaligned,Misaligned
4,Explain the overview of computers and programm...,Completed lab/written exercises. \nCompleted quiz,Recitation \nQuiz \nChapter exercises,Misaligned,Misaligned,Misaligned
5,Define and use arrays and describe how they oc...,Code Combat CS4 Completion Certificate\nComple...,Recitation \nLab/Written Exercises \nChapter e...,Misaligned,Aligned,Misaligned
6,Amazon Elastic Block Store (Amazon EBS)\nAmazo...,Case Study Presentation,Knowledge Check\nLab 6\nCase Study (Storage So...,Aligned,Misaligned,Misaligned
7,Amazon Relational Database Service (Amazon RDS...,Case Study Presentation,Knowledge Check Lab 7\nCase Study (Database So...,Misaligned,Misaligned,Misaligned
8,Explore input data for analysis\nApply basic f...,LinkedIn Learning Certificate,In-video quizzes\nProblem solving exercise,Misaligned,Misaligned,Misaligned
9,Determine ways in which the three key elements...,Certificates:\nModule 1: Data and Privacy\nMod...,Assessment: Show me what you know:\nModule 1: ...,Aligned,Misaligned,Misaligned
